# Neural Connections

After neural groups are defined, neural connections between different groups should be specified. In BrainPy, `brainpy.TwoEndConn` is used to model two-end synaptic connections. **All types of synaptic connections, build-in or user-customized, should inherit the `brainpy.TwoEndConn` base class.**

## brainpy.TwoEndConn

To define a synaptic connection class, three key elements should be given in the `__init__` function:
* `pre`: the presynaptic neural group. It should be a subclass of `brainpy.NeuGroup`.
* `post`: the postsynaptic neural group. It should be a subclass of `brainpy.NeuGroup`.
* `conn` (optional): the connection type between these two groups. Brainpy has provided abundant connection types that are described in details in the [Synaptic Connectivity](../tutorial_simulation/synaptic_connectivity.ipynb) section.

Besides, the `update(_t, dt)` function should also be implemented to update the synaptic state during dynamics simulation.

Here, let's illustrate how to use `brainpy.TwoEndConn` with the [Exponential synapse model](https://brainmodels.readthedocs.io/en/latest/apis/generated/brainmodels.synapses.ExpCOBA.html).

### Exponential Synapse Model

The exponential synapse model assumes that once a pre-synaptic neuron generates a spike, the synaptic state arises instantaneously, then decays with a certain time constant $\tau_{decay}$. Its dynamics is given by:

$$
\frac{d s}{d t} = -\frac{s}{\tau_{decay}}+\sum_{k} \delta(t-D-t^{k})
$$

where $s$ is the synaptic state, $t^{k}$ is the spike time of the pre-synaptic neuron, and $D$ is the synaptic delay. 

Afterward, the current output onto the post-synaptic neuron is given in the conductance-based form:

$$
I_{syn}(t) = g_{max} s \left( V(t)-E \right)
$$

where $E$ is the reversal potential of the synapse, $V$ is the post-synaptic membrane potential, $g_{max}$ is the maximum synaptic conductance. 

Below is the implementation of the exponential synapose model.

In [8]:
import brainpy as bp
import brainpy.math as bm

bm.set_platform('cpu')


class Exponential(bp.TwoEndConn):  # The exponential synapse should inherit bp.TwoEndConn
  def __init__(self, pre, post, conn, g_max=1., delay=0., tau=8.0, E=0., **kwargs):
    # connections are built in the initialization function of bp.TwoEndConn
    super(Exponential, self).__init__(pre=pre, post=post, conn=conn, **kwargs)

    # initialize parameters
    self.g_max = g_max
    self.E = E
    self.tau = tau
    self.delay = delay

    # acquire desired properties of the connection
    self.pre_ids, self.post_ids = self.conn.requires('pre_ids', 'post_ids')
    self.num = len(self.pre_ids)
    
    # initialize variables
    self.s = bm.Variable(bm.zeros(self.num))
    self.pre_spike = self.register_constant_delay('pre_spike', size=self.pre.num, delay=delay)
    
    # integral function
    self.integral = bp.odeint(self.derivative, method='exponential_euler')

  def derivative(self, s, t):
    dsdt = - s / self.tau
    return dsdt

  def update(self, _t, _dt):
    # push the pre-synaptic spikes into the delay
    self.pre_spike.push(self.pre.spike)
    
    # pull the delayed pre-synaptic spikes
    delayed_pre_spike = self.pre_spike.pull()
    
    
    spikes = bm.pre2syn(delayed_pre_spike, self.pre_ids)
    post_sp = bm.syn2post(spikes, self.post_ids, self.post.num)
    self.g.value = self.integral(self.g.value, _t, dt=_dt) + post_sp * self.g_max
    self.post.input.value += self.g
        
    # push the pre-synaptic spikes into the delay
    self.pre_spike.push(self.pre.spike)
    
    # pull the delayed pre-synaptic spikes
    delayed_pre_spike = self.pre_spike.pull()
    
    # update the synatic state
    self.s[:] = self.integral(self.s, _t)
    
    for syn_i in range(self.num):
      pre_i, post_i = self.pre_ids[syn_i], self.post_ids[syn_i]
    
      # P4: whether pre-synaptic neuron generates a spike
      if delayed_pre_spike[pre_i]:
        self.s[syn_i] += 1.
      
      # P5: output the synapse current onto the post-synaptic neuron
      self.post.input[post_i] += self.g_max * self.s[syn_i] * (self.E - self.post.V[post_i])

### Implementation Details

To make the contruction of connections easier, Brainpy provides `brainpy.connect.Connector` for standardized interface to build neural connections. **It is strongly suggested that users use build-in connectors or specify their customized connectors in `brainpy.connect.Connector` to generate different types of connections.** 

After defining the connector, users may want to access some useful properties of the connetion, such as the indices of pre- and post-synaptic neurons. ``brainpy.connect.Connector`` provides a `require` function by which users can request those properties like *pre_ids*, *post_ids*, *conn_mat*, *pre2post*, *post2pre*, *pre2syn*, *post2syn*, etc. In the above example, *pre_ids* and *post_ids*, i.e. the indices of pre- and post-synaptic neurons, are requested in the `require` function, so they will be computed and returned by the connector.

For more details, see [Synaptic Connectivity](../tutorial_building/synaptic_connectivity.ipynb).

The modeling of synapses usually includes a delay time (typically 0.3–0.5 ms) required for a neurotransmitter to be released from a presynaptic membrane, diffuse across the synaptic cleft, and bind to a receptor site on the post-synaptic membrane. ``brainpy.TwoEndConn`` can help to construct **automatic delay** in synaptic computations. `brainpy.DynamicalSystem`, the superclass for all dynamical systems including `brainpy.TwoEndConn`, provides the [register_constant_delay()](../apis/simulation/brainobjects.html#brainpy.simulation.brainobjects.DynamicalSystem.register_constant_delay) function for automatic state delay. In the above example, `self.pre_spike` is defined as a delayed variable.

## Building Connections between Neural Groups

For convenience, a LIF model is imported from the `BrainModels` package which provides plentiful canonical brain models backended by BrainPy, and the pre- and post-synaptic nerual groups are constructed.

In [5]:
from brainmodels.neurons import LIF

num_pre = 5
num_post = 4
pre_neu = LIF(num_pre, tau=10, V_th=-30, V_rest=-60, V_reset=-60, tau_ref=5.)
post_neu = LIF(num_post, tau=20, V_th=-30, V_rest=-60, V_reset=-60, tau_ref=5.)

The `Exponential` model can be instantiated as:

In [6]:
conn = bp.connect.All2All()  # all-to-all connections, a subclass of bp.connect.Connector
exp_syn = Exponential(pre_neu, post_neu, conn, E=0., g_max=0.6, tau=5)

Here the all-to-all connection type is used, which connects each presynaptic neuron to all postsynaptic neurons. Because the `pre_ids` and `post_ids` have been **required** in the `Exponential` class, they can be accessed directly using `exp_syn.pre_ids` and `exp_.post_ids`.

In [7]:
import numpy as np

print('indices of synapses:{}'.format(np.arange(num_pre * num_post)))
print('indices of presynaptic neurons: {}'.format(exp_syn.pre_ids))
print('indices of postsynaptic neurons: {}'.format(exp_syn.post_ids))

indices of synapses:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
indices of presynaptic neurons: [0 0 0 0 1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4]
indices of postsynaptic neurons: [0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3]


After neural groups and neural connections are constructed, a neural network can be built for dynamic simulation. Please see the next part [Neural Networks](neural_networks.ipynb).